In [1]:
import os
import requests

Downloading The Document (PDF File)

In [2]:
# get document name
pdf_name = "human-nutrition-text.pdf"

if not os.path.exists(pdf_name):
    print(f"Downloading {pdf_name}")

    # url of document to download

    pdf_url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

    resp = requests.get(pdf_url)

    if resp.status_code == 200:
        with open(pdf_name, "wb") as f:
            f.write(resp.content)
        print(f"Downloaded {pdf_name} and Saved")
    else:
        print(f"Failed to download {pdf_name}")
else:
    print(f"{pdf_name} already exists")


human-nutrition-text.pdf already exists


Open the PDF

In [3]:
!pip install PyMuPDF

In [4]:
!pip install spacy

In [5]:
import fitz  # PyMuPDF !pip install pymupdf
from tqdm.auto import tqdm

def text_formatter(text:str)->str:
    """perform basic text preprocessing. """

    cleaned_text = text.replace("\n", " ").strip()

    return cleaned_text


def open_and_read_pdf(pdf_name:str)->list[dict]:
    doc = fitz.open(pdf_name)
    pages_and_textx = []

    for page_no,page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text)
        pages_and_textx.append({
            "page_number" : page_no+1,
            "page_char_count" : len(text),
            "page_word_count" : len(text.split()),
            "page_sentence_count_raw" : len(text.split(". ")),
            "page_token_count" : len(text)/4,
            "text" : text
            })
    return pages_and_textx

pages_and_texts = open_and_read_pdf(pdf_name)

d:\Coding\FileChat-Using-RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
1208it [00:04, 265.74it/s]


In [6]:
import random

random.sample(pages_and_texts,k=3)

[{'page_number': 265,
  'page_char_count': 2161,
  'page_word_count': 330,
  'page_sentence_count_raw': 18,
  'page_token_count': 540.25,
  'text': 'drinking sports drinks instead of plain water enhances endurance  or performance in individuals exercising less than one hour and at  low to moderate intensities. A well-concocted sports drink contains  sugar, water, and sodium in the correct proportions so that  hydration is optimized. The sugar is helpful in maintaining blood- glucose levels needed to fuel muscles, the water keeps an athlete  hydrated, and the sodium enhances fluid absorption and replaces  some of that lost in sweat. The American College of Sports Medicine  states that the goal of drinking fluids during exercise is to prevent  dehydration, which compromises performance and endurance.  The primary source of water loss during intense physical activity  is sweat. Perspiration rates are variable and dependent on many  factors including body composition, humidity, temperature

In [7]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,1,29,4,1,7.25,Human Nutrition: 2020 Edition
1,2,0,0,1,0.00,
2,3,320,42,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,4,212,30,1,53.00,Human Nutrition: 2020 Edition by University of...
4,5,797,116,2,199.25,Contents Preface University of Hawai‘i at Mā...


In [8]:
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000
mean,604.50000,1148.004139,171.966060,9.972682,287.001035
std,348.86387,560.382275,86.491465,6.187226,140.095569
min,1.00000,0.000000,0.000000,1.000000,0.000000
25%,302.75000,762.000000,109.000000,4.000000,190.500000
50%,604.50000,1231.500000,183.000000,10.000000,307.875000
75%,906.25000,1603.500000,239.000000,14.000000,400.875000
max,1208.00000,2308.000000,393.000000,32.000000,577.000000


Why we need token Count?

1. Embeding Model don't deal with infinite tokens.
2. LLMs don't deal with lnfinite tokens.

# Splitting Pages into sentences

Two days to do this
1. split by `". "` OR `".\n"`
2. Use NLP like spaCy or NLTK

In [9]:
from spacy.lang.en import English 

nlp = English()

# add a sentencilizer pipeline 
nlp.add_pipe("sentencizer")

# create a document example
doc = nlp("THis is sentence1. I like cats")

list(doc.sents)

In [14]:
pages_and_texts[600]

{'page_number': 601,
 'page_char_count': 863,
 'page_word_count': 117,
 'page_sentence_count_raw': 8,
 'page_token_count': 215.75,
 'text': 'Image by  Allison  Calabrese /  CC BY 4.0  Korsakoff syndrome can cause similar symptoms as beriberi such  as confusion, loss of coordination, vision changes, hallucinations,  and may progress to coma and death. This condition is specific  to alcoholics as diets high in alcohol can cause thiamin deficiency.  Other individuals at risk include individuals who also consume diets  typically low in micronutrients such as those with eating disorders,  elderly, and individuals who have gone through gastric bypass  surgery.5  Figure 9.10 The Role of Thiamin  Figure 9.11 Beriberi, Thiamin Deficiency  5.\xa0Fact Sheets for Health Professionals: Thiamin. National  Institute of Health, Office of Dietary Supplements.  \xa0https://ods.od.nih.gov/factsheets/Thiamin- HealthProfessional/. Updated Feburary 11, 2016.  Accessed October 22, 2017.  Water-Soluble Vitami

In [16]:
from tqdm import tqdm

for page in tqdm(pages_and_texts):
    page["sentences"] = [str(sent) for sent in nlp(page['text']).sents]
    
    page["page_sentence_count"] = len(page["sentences"])

100%|██████████| 1208/1208 [00:06<00:00, 183.88it/s]


In [20]:
pages_and_texts[103]

{'page_number': 104,
 'page_char_count': 1693,
 'page_word_count': 264,
 'page_sentence_count_raw': 13,
 'page_token_count': 423.25,
 'text': 'Basic Biology, Anatomy, and  Physiology  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  The Basic Structural and Functional Unit of Life:  The Cell  What distinguishes a living\xa0organism from an inanimate object? A  living organism conducts self-sustaining biological processes. A cell  is the smallest and most basic form of life.  The cell theory incorporates three principles:  Cells are the most basic building units of life.\xa0All living things  are composed of cells. New cells are made from preexisting cells,  which divide in two. Who you are has been determined because  of two cells that came together inside your mother’s womb. The  two cells containing all of your genetic information (DNA) united to  begin making new life. Cells divided and differentiated into other  cells with specif

In [21]:
df = pd.DataFrame(pages_and_texts)
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000,1208.000000
mean,604.50000,1148.004139,171.966060,9.972682,287.001035,10.319536
std,348.86387,560.382275,86.491465,6.187226,140.095569,6.300843
min,1.00000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,302.75000,762.000000,109.000000,4.000000,190.500000,5.000000
50%,604.50000,1231.500000,183.000000,10.000000,307.875000,10.000000
75%,906.25000,1603.500000,239.000000,14.000000,400.875000,15.000000
max,1208.00000,2308.000000,393.000000,32.000000,577.000000,28.000000


## Chunking the sentences together

here we are getting in 10 sentences at a time.

Framework like langchain can do this task easily.

## Need of Chunking
1. makes easy to filter 
2. context of embedding model 
2. context of llm

In [23]:
# define number of sentences chunk
sentences_chunk_size = 10

# actual function to split list of sentences to list of list of size <= 10 of sentences

def split_sentence_chunk(sentence_list:list[str],size_chunk:int = sentences_chunk_size) -> list[list[str]]:
    return [sentence_list[i:i + size_chunk] for i in range(0,len(sentence_list),size_chunk)]

test  = list(range(1,26))
print(test)
split_sentence_chunk(test)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]


[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
 [21, 22, 23, 24, 25]]

In [24]:
# make chunks of original data
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_sentence_chunk(item["sentences"],sentences_chunk_size)
    item["chunk_count"] = len(item["sentence_chunks"])

100%|██████████| 1208/1208 [00:00<00:00, 4013.06it/s]


In [26]:
random.sample(pages_and_texts,k=1)

[{'page_number': 580,
  'page_char_count': 1444,
  'page_word_count': 207,
  'page_sentence_count_raw': 12,
  'page_token_count': 361.0,
  'text': 'Cardiovascular Disease  Vitamin E reduces the oxidation of LDLs, and it was therefore  hypothesized that vitamin E supplements would protect against  atherosclerosis. However, large clinical trials have not consistently  found evidence to support this hypothesis. In fact, in the “Women’s  Angiographic Vitamin and Estrogen Study,” postmenopausal women  who took 400 international units (264 milligrams) of vitamin E and  500 milligrams of vitamin C twice per day had higher death rates  from all causes.7  Other studies have not confirmed the association between  increased vitamin E intake from supplements and increased  mortality. There is more consistent evidence from observational  studies that a higher intake of vitamin E from foods is linked to a  decreased risk of dying from a heart attack.  Cancer  The large clinical trials that evaluated

In [27]:
df=pd.DataFrame(pages_and_texts)
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count,chunk_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000,1208.000000,1208.000000
mean,604.50000,1148.004139,171.966060,9.972682,287.001035,10.319536,1.525662
std,348.86387,560.382275,86.491465,6.187226,140.095569,6.300843,0.644397
min,1.00000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,302.75000,762.000000,109.000000,4.000000,190.500000,5.000000,1.000000
50%,604.50000,1231.500000,183.000000,10.000000,307.875000,10.000000,1.000000
75%,906.25000,1603.500000,239.000000,14.000000,400.875000,15.000000,2.000000
max,1208.00000,2308.000000,393.000000,32.000000,577.000000,28.000000,3.000000


## Splitting chunks to separate items
so that we can add meta data to each chunk ans can embed them together.


In [29]:
pages_and_chunks = []

for item in tqdm(pages_and_texts):
    for chunk in item["sentence_chunks"]:
        chunk_dict = {}
            
        chunk_dict["page_number"] = item["page_number"] # original page number
        chunk_dict["text"] =  " ".join(chunk).replace("  "," ").strip() # original text
        chunk_dict["sentence_chunk"] = chunk
        chunk_dict["chunk_char_count"]  =  len(chunk_dict["text"])
        chunk_dict["chunk_word_count"]  =  len([word for word in chunk_dict["text"].split() if word])
        chunk_dict["chunk_token_count"] = chunk_dict["chunk_char_count"] / 4
        
        pages_and_chunks.append(chunk_dict)
        
len(pages_and_chunks)      

  0%|          | 0/1208 [00:00<?, ?it/s]

100%|██████████| 1208/1208 [00:00<00:00, 4467.75it/s]


1843

In [33]:
random.sample(pages_and_chunks,k=1)

[{'page_number': 210,
  'text': 'Image by Allison Calabrese / CC BY 4.0 The Kidneys Detect Blood Volume The kidneys are two bean-shaped organs, each about the size of a fist and located on either side of the spine just below the rib cage. The kidneys filter about 190 liters of blood and produce (on average) 1.5 liters of urine per day. Urine is mostly water, but it also contains electrolytes and waste products, such as urea. The amount of water filtered from the blood and excreted as urine is dependent on the amount of water in, and the electrolyte composition in the blood. Kidneys have protein sensors that detect blood volume from the pressure, or stretch, in the blood vessels of the kidneys. When blood volume is low, kidney cells detect decreased pressure and secrete the enzyme, renin. Renin travels in the blood and cleaves another protein into the active hormone, angiotensin. Angiotensin targets three different organs (the adrenal glands, the hypothalamus, and 168 | Regulation of Wa

In [34]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1843.00,1843.00,1843.00,1843.00
mean,625.38,735.11,112.85,183.78
std,347.79,447.64,71.28,111.91
min,1.00,12.00,3.00,3.00
25%,322.50,315.00,45.00,78.75
50%,628.00,747.00,115.00,186.75
75%,932.00,1119.00,173.00,279.75
max,1208.00,1832.00,298.00,458.00


# remove chunks with less than 30 token

In [44]:
min_chunk_token_limit = 30

pages_and_chunks_filtered = df[df['chunk_token_count'] > min_chunk_token_limit].to_dict(orient="records")

In [60]:
random.sample(pages_and_chunks_filtered,k=1)

[{'page_number': 643,
  'text': 'Phytochemical Phytochemical Source Phytochemical Function: Carotenoid Yellow-orange fruits, dark green leafy vegetables May possess strong cancer-fighting properties Indoles Cruciferous vegetables (i.e. bok choy, broccoli, choy sum) May inhibit the development of cancer-causing hormones and prevent tumor growth Phytoestrogen Grapes, berries, plums, soybeans, tofu, garlic May lower the risk for osteoporosis, heart disease, breast cancer, and menopausal symptoms Stanols Grains, nuts, legumes May lower blood cholesterol levels and reduce the risk of heart disease and stroke Saponins Broad beans, kidney beans, lentils May decrease blood lipids, lower cancer risks, and lower blood glucose response Terpenes Citrus fruits May slow cancer cell growth, aid in immune system support, and prevent virus related illness Flavonoids Fruits, vegetables, chocolates, wines, teas, nuts, seeds May benefit the immune system and prevent cancer cell growth. Anthocyanidins Frui